experimental: i am not even sure why you would use this.

In [1]:
    try:
        from . import template
    except:
        import template
    from traitlets import Any, observe
    from types import MethodType
    from contextlib import contextmanager
    from IPython.core.interactiveshell import InteractiveShell
    from IPython import display, get_ipython
    from ipywidgets import HTML, Widget
    from mistune import markdown
    import ipywidgets

In [2]:
    class Template(HTML):
        template = Any()
        ns = Any(default_value=dict())
        @observe('template')
        def _change_render(self, change=dict()):
            try:
                self.value = markdown(self.template.render(**self.ns))
            except Exception as e: ...

In [3]:
    class Interact(template.Jinja2):
        def macro(self, body, *, ns=dict(), out=tuple()):
            return Template(template=body, ns=ns),        

In [4]:
    @contextmanager
    def _run_cell(self, raw_cell, store_history=False, silent=False, shell_futures=True):
        from ipywidgets import Widget
        yield InteractiveShell.run_cell(self, raw_cell, store_history=False, silent=False, shell_futures=True)
        for value in Widget.widgets.values():
            if isinstance(value, Template): value._change_render()

    def run_cell(self, raw_cell, store_history=False, silent=False, shell_futures=True):
        with _run_cell(self, raw_cell, store_history=False, silent=False, shell_futures=True) as e:
            ...
        return e

In [5]:
    def load_ipython_extension(ip=get_ipython()):
        template.extension(Interact)(ip)
        ip.run_cell = MethodType(run_cell, get_ipython())

In [6]:
    def unload_ipython_extension(ip=get_ipython()):
        ip.run_cell = MethodType(InteractiveShell.run_cell, get_ipython())
        template.unload_ipython_extension()

In [7]:
    if __name__ == '__main__':
        load_ipython_extension()
        !jupyter nbconvert --to python --TemplateExporter.exclude_input_prompt=True widget.ipynb

[NbConvertApp] Converting notebook widget.ipynb to python
[NbConvertApp] Writing 1869 bytes to widget.py
